importing libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import Binarizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb
from sklearn.linear_model import Lasso
import lightgbm as lgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
# Load the dataset
file_path = "/content/ecommerce_price_predictor_combined_with_sales_discounted_final.xlsx"
df = pd.read_excel(file_path, sheet_name=0)
print("Data Preview:")
print(df.head())

Data Preview:
      Product Name    Category  Price (USD)  Rating Stock Availability  \
0  Sony WH-1000XM5  Headphones       112.85     3.7           In Stock   
1  Sony WH-1000XM5  Headphones       112.85     3.7           In Stock   
2  Sony WH-1000XM5  Headphones       112.85     3.7           In Stock   
3  Sony WH-1000XM5  Headphones       112.85     3.7           In Stock   
4  Sony WH-1000XM5  Headphones       112.85     3.7           In Stock   

   Discount (%) Brand  Reviews Count  Shipping Time (days) Return Policy  ...  \
0            12  Sony           2745                     8    No returns  ...   
1            12  Sony           2745                     8    No returns  ...   
2            12  Sony           2745                     8    No returns  ...   
3            12  Sony           2745                     8    No returns  ...   
4            12  Sony           2745                     8    No returns  ...   

   Competitor Price Difference (%) Profit Margin (%)  

data processing and cleaning

In [ ]:
# Removing duplicates
df.drop_duplicates(inplace=True)
# Check for missing values
print(df.isnull().sum())

Product Name                       0
Category                           0
Price (USD)                        0
Rating                             0
Stock Availability                 0
Discount (%)                       0
Brand                              0
Reviews Count                      0
Shipping Time (days)               0
Return Policy                      0
Demand Index                       0
Seasonality Impact                 0
Competitor Price Difference (%)    0
Profit Margin (%)                  0
Customer Satisfaction (%)          0
Supply Chain Efficiency (%)        0
Competitor Name                    0
Competitor Price (USD)             0
Competitor Rating                  0
Competitor Stock Availability      0
Sales (Units Sold)                 0
Discounted Price (USD)             0
dtype: int64


In [ ]:
df.shape

(39424, 22)

In [ ]:
df.columns

Index(['Product Name', 'Category', 'Price (USD)', 'Rating',
       'Stock Availability', 'Discount (%)', 'Brand', 'Reviews Count',
       'Shipping Time (days)', 'Return Policy', 'Demand Index',
       'Seasonality Impact', 'Competitor Price Difference (%)',
       'Profit Margin (%)', 'Customer Satisfaction (%)',
       'Supply Chain Efficiency (%)', 'Competitor Name',
       'Competitor Price (USD)', 'Competitor Rating',
       'Competitor Stock Availability', 'Sales (Units Sold)',
       'Discounted Price (USD)'],
      dtype='object')

In [ ]:
# Handling outliers using IQR
Q1 = df.quantile(0.25, numeric_only=True)
Q3 = df.quantile(0.75, numeric_only=True)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter numerical columns for outlier removal
numerical_cols = df.select_dtypes(include=np.number).columns

# Removing outliers only from numerical columns
df_filtered = df[~((df[numerical_cols] < lower_bound[numerical_cols]) | (df[numerical_cols] > upper_bound[numerical_cols])).any(axis=1)]

df = df_filtered

In [ ]:
df.dropna()

,Product Name,Category,Price (USD),Rating,Stock Availability,Discount (%),Brand,Reviews Count,Shipping Time (days),Return Policy,...,Competitor Price Difference (%),Profit Margin (%),Customer Satisfaction (%),Supply Chain Efficiency (%),Competitor Name,Competitor Price (USD),Competitor Rating,Competitor Stock Availability,Sales (Units Sold),Discounted Price (USD)
0,Sony WH-1000XM5,Headphones,112.85,3.7,In Stock,12,Sony,2745,8,No returns,...,-0.50,36.59,74,93,Newegg,113.42,3.4,In Stock,4010,99.3080
1,Sony WH-1000XM5,Headphones,112.85,3.7,In Stock,12,Sony,2745,8,No returns,...,-0.50,36.59,74,93,Bose Store,1228.90,4.1,Out of Stock,1465,99.3080
2,Sony WH-1000XM5,Headphones,112.85,3.7,In Stock,12,Sony,2745,8,No returns,...,-0.50,36.59,74,93,Amazon,147.94,3.5,In Stock,119,99.3080
3,Sony WH-1000XM5,Headphones,112.85,3.7,In Stock,12,Sony,2745,8,No returns,...,-0.50,36.59,74,93,Adidas Store,2142.34,3.9,In Stock,3813,99.3080
4,Sony WH-1000XM5,Headphones,112.85,3.7,In Stock,12,Sony,2745,8,No returns,...,-0.50,36.59,74,93,Apple Store,1608.89,4.3,Out of Stock,1436,99.3080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39419,Philips Prestige Hair Straightener,Beauty,996.38,4.7,In Stock,24,Philips,802,5,No returns,...,4.68,25.51,94,77,Walmart,1028.71,4.3,Out of Stock,60,757.2488
39420,Philips Prestige Hair Straightener,Beauty,996.38,4.7,In Stock,24,Philips,802,5,No returns,...,4.68,25.51,94,77,Apple Store,112.02,4.5,Out of Stock,702,757.2488
39421,Philips Prestige Hair Straightener,Beauty,996.38,4.7,In Stock,24,Philips,802,5,No returns,...,4.68,25.51,94,77,BestBuy,852.08,4.1,Out of Stock,537,757.2488
39422,Philips Prestige Hair Straightener,Beauty,996.38,4.7,In Stock,24,Philips,802,5,No returns,...,4.68,25.51,94,77,Newegg,1835.96,5.2,In Stock,1459,757.2488


In [ ]:
df.count()

,0
Product Name,39424
Category,39424
Price (USD),39424
Rating,39424
Stock Availability,39424
Discount (%),39424
Brand,39424
Reviews Count,39424
Shipping Time (days),39424
Return Policy,39424


#models

In [ ]:
#feature selection
X = df[['Price (USD)', 'Sales (Units Sold)', 'Competitor Price (USD)', 'Stock Availability', 'Competitor Stock Availability']].copy()
y = df['Discounted Price (USD)'].copy()

In [ ]:
# Convert categorical variables to numerical using one-hot encoding
X = pd.get_dummies(X, drop_first=True)

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Linear Regression
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)

In [ ]:
# Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

In [ ]:
# SVM
svm_model = SVR()
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)

In [ ]:
# XGBoost
xgb_model = XGBRegressor(objective='reg:squarederror', n_estimators=100, max_depth=5, learning_rate=0.1)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

In [ ]:
# AdaBoost
ab_model = AdaBoostRegressor(n_estimators=100, random_state=42)
ab_model.fit(X_train, y_train)
y_pred_ab = ab_model.predict(X_test)

In [ ]:
# Results
results = [
    {'Model': 'Linear Regression', 'MAE': mean_absolute_error(y_test, y_pred_lr), 'MSE': mean_squared_error(y_test, y_pred_lr), 'RMSE': np.sqrt(mean_squared_error(y_test, y_pred_lr)), 'R2 Score': r2_score(y_test, y_pred_lr)},
    {'Model': 'Random Forest', 'MAE': mean_absolute_error(y_test, y_pred_rf), 'MSE': mean_squared_error(y_test, y_pred_rf), 'RMSE': np.sqrt(mean_squared_error(y_test, y_pred_rf)), 'R2 Score': r2_score(y_test, y_pred_rf)},
    {'Model': 'SVM', 'MAE': mean_absolute_error(y_test, y_pred_svm), 'MSE': mean_squared_error(y_test, y_pred_svm), 'RMSE': np.sqrt(mean_squared_error(y_test, y_pred_svm)), 'R2 Score': r2_score(y_test, y_pred_svm)},
    {'Model': 'XGBoost', 'MAE': mean_absolute_error(y_test, y_pred_xgb), 'MSE': mean_squared_error(y_test, y_pred_xgb), 'RMSE': np.sqrt(mean_squared_error(y_test, y_pred_xgb)), 'R2 Score': r2_score(y_test, y_pred_xgb)},
    {'Model': 'AdaBoost', 'MAE': mean_absolute_error(y_test, y_pred_ab), 'MSE': mean_squared_error(y_test, y_pred_ab), 'RMSE': np.sqrt(mean_squared_error(y_test, y_pred_ab)), 'R2 Score': r2_score(y_test, y_pred_ab)}
]

# Display results
results_df = pd.DataFrame(results)
print(results_df.sort_values(by='R2 Score', ascending=False))

               Model         MAE           MSE        RMSE  R2 Score
1      Random Forest    0.256584     14.531372    3.812004  0.999972
3            XGBoost  104.449957  18352.567226  135.471647  0.964384
0  Linear Regression  123.934085  25835.993293  160.735787  0.949861
4           AdaBoost  135.927242  27351.388425  165.382552  0.946920
2                SVM  152.100378  39924.986987  199.812379  0.922519
